# RSS: Expansión

## 1. Import Libraries

In [46]:
import feedparser
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import googletrans
import datetime
from termcolor import colored
from IPython.display import display, Markdown, Latex, HTML
import re
from IPython.display import Image
import urllib

In [37]:
def get_only_text1(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    # Eliminar Última Hora
    rest = soup.find_all('p', attrs={'class' : "line-clamp_x2 ue-c-widget__article-headline"})[0].text
    index = text.find(rest)
    final_text = text[:index]
    return [final_text,soup.title.text]

def get_only_text(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    return [text,soup.title.text]

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [40]:
economista_entries[1]['id']

'http://www.eleconomista.es/empresas-finanzas/noticias/10652645/07/20/Abalos-senala-que-solo-se-rescatara-a-las-aerolineas-que-lo-soliciten.html'

In [20]:
economista_entries[0]

{'title': 'El mercado del plástico reciclado duplicará su valor hasta el año 2027',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://www.eleconomista.es/rss/rss-empresas.php',
  'value': 'El mercado del plástico reciclado duplicará su valor hasta el año 2027'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'http://www.eleconomista.es/empresas-finanzas/noticias/10652734/07/20/El-mercado-del-plastico-reciclado-duplicara-su-valor-hasta-el-ano-2027.html'}],
 'link': 'http://www.eleconomista.es/empresas-finanzas/noticias/10652734/07/20/El-mercado-del-plastico-reciclado-duplicara-su-valor-hasta-el-ano-2027.html',
 'summary': 'Las empresas apuestan por potenciar el mercado del plástico reciclado, cuyo valor crecerá a un ritmo del 6,8% anual, desde los 30.000 millones de euros registrados en 2019 a los 53.000 millones previstos para 2027, según el informe La ambición empresarial para avanzar hacia la nueva economía de plásticos, presentado est

In [50]:
def economista_news(economista_entries,word_count=100):
    economista = {}
    for i in range(len(economista_entries)):
        noticia = {'newspaper':None,'title':None,'headline':None,'summarise':None,'date':None,'link':None,'text':None,'current_date':None,'img':None,'premium':None}
        # Periodico
        noticia['newspaper'] = find_between(s=economista_entries[i]['id'], first='www.el', last='.' )
        # Title
        noticia['title'] = economista_entries[i]['title']
        # Headline
        noticia['headline'] = economista_entries[i]['summary']
        # Date
        noticia['date'] = {'date1':economista_entries[i]['published'],'date2':economista_entries[i]['published_parsed']}
        # Link
        noticia['link'] = economista_entries[i]['id']
        # Text
        text = get_only_text(economista_entries[i]['id'])[0]
        noticia['text'] = text
        # Summarise
        noticia['summarise'] = summarize(text, word_count=word_count)
        # Current Date
        noticia['current_date'] = datetime.datetime.now().timetuple()
        # Image
        noticia['img'] = None
        economista.update({'Noticia'+'_'+str("%02d")%(i+1):noticia})
    return economista

## 2. RSS Expansión

In [51]:
url_economista = 'https://www.eleconomista.es/rss/rss-empresas.php'
economista = feedparser.parse(url_economista)
economista_entries = economista['entries']

print('Total Entradas Economista:',len(economista_entries))

Total Entradas Economista: 20


In [52]:
economista = economista_news(economista_entries,word_count=100)

In [53]:
economista

{'Noticia_01': {'newspaper': 'economista',
  'title': 'El mercado del plástico reciclado duplicará su valor hasta el año 2027',
  'headline': 'Las empresas apuestan por potenciar el mercado del plástico reciclado, cuyo valor crecerá a un ritmo del 6,8% anual, desde los 30.000 millones de euros registrados en 2019 a los 53.000 millones previstos para 2027, según el informe La ambición empresarial para avanzar hacia la nueva economía de plásticos, presentado este martes por Forética.',
  'summarise': 'Las empresas apuestan por potenciar el mercado del plástico reciclado, cuyo valor crecerá a un ritmo del 6,8% anual, desde los 30.000 millones de euros registrados en 2019 a los 53.000 millones previstos para 2027, según el informe La ambición empresarial para avanzar hacia la nueva economía de plásticos, presentado este martes por Forética.\nLas empresas apuestan por potenciar el mercado del plástico reciclado, cuyo valor crecerá a un ritmo del 6,8% anual, desde los 30.000 millones de euro

In [64]:
yesterday = datetime.datetime.today() - datetime.timedelta(days=1)
before_yes = datetime.datetime.today() - datetime.timedelta(days=2)

for i in list(expansion.keys()):
    if expansion[i]['date']['date2'][2] in [datetime.datetime.today().day,yesterday.day,before_yes.day]:
        print(colored((expansion[i]['newspaper']).upper(),'green',attrs=['bold']))
        print(colored(expansion[i]['date']['date1'],'grey',attrs=['bold']))
        print(colored(expansion[i]['title'],'magenta',attrs=['bold']))
        print(colored('Headline','blue'))
        print(expansion[i]['headline'])
        print(colored('Summarise','blue'))
        
        if expansion[i]['summarise'] != 'Premium Content':
            print(expansion[i]['summarise'])
        else:
            print(colored(expansion[i]['summarise'],'red'))
        print(colored('Text','blue'))
        print(expansion[i]['text'])
        url = expansion[i]['img']
        display(Image(url=url,width=500))
        display(Markdown('<a href="'+expansion[i]['link']+'">Link</a>'))
        print('\n')

EXPANSION
Wed, 01 Jul 2020 02:19:08 +0200
Catalana Occidente recorta su cobertura de seguro de crédito
Headline
Las pymes ven reducido el respaldo del seguro para cerrar operaciones comerciales ante el previsible aumento de las insolvencias.
Summarise
Premium Content
Text
Las pymes ven reducido el respaldo del seguro para cerrar operaciones comerciales ante el previsible aumento de las insolvencias. Catalana Occidente está reduciendo la cobertura que da a sus clientes en el seguro de crédito, tras el estallido de la crisis del Covid-19.
Este recorte de la protección, -del 15% de media, según fuentes del sector- no es exclusivo de Catalana Occid Para seguir leyendo, hágase Premium ¿Ya es Premium? Inicie sesión Mensual primer mes después 14,99€   Anual primer mes después 7,50€/mes (Pago único de 90€)
 4 meses gratis  Descubra qué es Premium   Cancele cuando quiera    Consulte los términos y condiciones del servicio  


<a href="https://www.expansion.com/empresas/banca/2020/07/01/5efba8e1468aebde618b45fe.html">Link</a>



EXPANSION
Wed, 01 Jul 2020 02:18:39 +0200
La apuesta de Intesa por España
Headline
La entidad quiere ampliar su base de clientes más allá de las grandes empresas españolas.
Summarise
Premium Content
Text
La entidad quiere ampliar su base de clientes más allá de las grandes empresas españolas. Intesa Sanpaolo quiere crecer en banca de inversión en España. Y ahora cuenta con dos banqueros más para conseguir ese objetivo.
El responsable de la sucursal española, Juan Pontoni, ha fichado a Javier Vidal como director de Originación de DCM (sig Para seguir leyendo, hágase Premium ¿Ya es Premium? Inicie sesión Mensual primer mes después 14,99€   Anual primer mes después 7,50€/mes (Pago único de 90€)
 4 meses gratis  Descubra qué es Premium   Cancele cuando quiera    Consulte los términos y condiciones del servicio  


<a href="https://www.expansion.com/empresas/banca/2020/07/01/5efba7c6468aeb76208b4625.html">Link</a>



EXPANSION
Wed, 01 Jul 2020 01:10:48 +0200
El Covid tendrá un impacto fuerte en los balances de la banca
Headline

Summarise
El Banco de España está convencido de que esta crisis, a diferencia de la de 2008, ha cogido al sector financiero con una buena posición de capital, a pesar de que su ratio de solvencia esté por debajo del promedio europeo.
En este sentido, advierte también que la recuperación de los créditos dudosos y su posterior venta (al capital riesgo) puede ser "más complejo durante un tiempo".
Durante marzo y abril, la inyección de financiación bancaria a las empresas para cubrir necesidades de liquidez ha sido de 98.000 millones de euros.
Text
 El Banco de España está convencido de que esta crisis, a diferencia de la de 2008, ha cogido al sector financiero con una buena posición de capital, a pesar de que su ratio de solvencia esté por debajo del promedio europeo. Pero no oculta que esta recesión provocará un súbito aumento de la morosidad que afectará a los balances.  "

...

<a href="https://www.expansion.com/empresas/banca/2020/06/29/5ef9bc29e5fdea35248b45df.html">Link</a>



EXPANSION
Mon, 29 Jun 2020 11:38:10 +0200
El Banco de España 'congela' el colchón anticíclico hasta que el impacto del Covid se disipe
Headline
El Banco de España ha decidido no activar el colchón anticíclico que podía exigir a las entidades por su exposición crediticia ante la situación creada por el Covid-19 y lo mantendrá así "por un período de tiempo prolongado" o, al menos, "hasta que los principales efectos económicos y financieros derivados de la crisis del coronavirus se hayan disipado y se haya retornado a una senda de recuperación".
Summarise
El Banco de España ha decidido no activar el colchón anticíclico que podía exigir a las entidades por su exposición crediticia ante la situación creada por el Covid-19 y lo mantendrá así "por un período de tiempo prolongado" o, al menos, "hasta que los principales efectos económicos y financieros derivados de la crisis del coronavirus se hayan disipado y se haya retornado a una senda de recuperación".
En la actual coyuntura el Banco de

<a href="https://www.expansion.com/empresas/banca/2020/06/29/5ef9b678e5fdea853c8b4626.html">Link</a>



EXPANSION
Mon, 29 Jun 2020 10:37:07 +0200
Andbank se alía con Inveready para ofrecer fondos de capital riesgo a sus clientes
Headline
El banco contará con un periodo de exclusividad en la comercialización de aquellos proyectos de inversión de Inveready que considere que pueden aportar valor a las carteras de los clientes de banca privada.
Summarise
El banco contará con un periodo de exclusividad en la comercialización de aquellos proyectos de inversión de Inveready que considere que pueden aportar valor a las carteras de los clientes de banca privada.
Andbank España e Inveready han llegado a un acuerdo por el cual la sociedad gestora ofrecerá a los clientes de Andbank la inversión en los fondos de capital riesgo gestionados por Inveready.
Andbank España contará con un periodo de exclusividad en la comercialización de aquellos proyectos de inversión de Inveready que considere que pueden aportar valor a las carteras de los clientes.
Text
El banco contará con un periodo de exclusividad 

<a href="https://www.expansion.com/catalunya/2020/06/29/5ef9a827e5fdea37498b45b4.html">Link</a>